In [1]:
import pandas as pd
import numpy as np
from random import random

from CTL2.causal_tree_learn import CausalTree
from DGP.DGP import dgp2
from DGP.DGP import dgp2train
from DGP.DGP import dgp2test
from DGP.DGP import dgp4
from DGP.DGP import dgp4int
from DGP.DGP import dgp8

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse

import matplotlib.pyplot as plt
import pickle as pkl
#from notify_run import Notify 
#notify = Notify()
import datetime

from utility.utility import transposer as tp
from utility.utility import writer
from utility.utility import reader

### Parameters:

In [6]:
def mc_ate2_test(n, tsize, var_e, reps, nomin_test):
    #true ATE
    tauhat = np.zeros((tsize, 1))
    #true ITE
    x_test, y_test, treat_test = dgp2test(tsize,var_e)
    for i in range(reps):
        x_train, y_train, treat_train = dgp2train(n, var_e)
        tau = np.where((x_test[:,0] >= 0),1.5, -1.5)
        ctl = CausalTree(honest=True, weight=0.0, split_size=0.0, max_depth = 3) #which type of tree to call
        ctl.fit(x_train, y_train, treat_train, nomin_test = nomin_test) #select est size when fitting
        ctl.prune()
        ctl_predict = ctl.predict(x_test)
        #predicted ATE
        tauhat = np.append(tauhat, ctl_predict.reshape(-1,1), axis = 1)

    #TOTAL MSE's
    ind_var = np.var(tauhat[:, 1:], axis = 1)
    ind_mean = np.mean(tauhat, axis = 1)

    total_var = np.sum(ind_var)/tsize
    total_bias = np.sum(np.square(np.subtract(ind_mean, tau)))/tsize
    total_mse = total_var + total_bias
    results = np.concatenate([np.array([nomin_test]), np.array([total_bias]), np.array([total_var]), np.array([total_mse])])

    return results

### ATE2

In [3]:
def mc_ate2(n, tsize, var_e, reps, nomin_test):
    #true ATE
    ATE1 = np.ones(reps)*1.5
    ATE2 = np.ones(reps)*-1.5
    #predicted ATE
    ate1 = []
    ate2 = []
    #ITE dummy
    tauhat = np.zeros((tsize, 1))
    #true ITE

    for i in range(reps):
        x_train, x_test, y_train, y_test, treat_train, treat_test = dgp2(n, tsize, var_e)
        tau = np.where((x_test[:,0] >= 0),1.5, -1.5)
        ctl = CausalTree(honest=True, weight=0.0, split_size=0.0, max_depth = 3) #which type of tree to call
        ctl.fit(x_train, y_train, treat_train, nomin_test = nomin_test) #select est size when fitting
        ctl.prune()
        ctl_predict = ctl.predict(x_test)
        #predicted ATE
        ate1 = np.append(ate1, np.mean(ctl_predict[np.where(x_test[:,0] >= 0)]))
        ate2 = np.append(ate2, np.mean(ctl_predict[np.where(x_test[:,0] < 0)]))
        #predicted ITE
        tauhat = np.append(tauhat, ctl_predict.reshape(-1,1), axis = 1)

    mean_ate = np.array([np.mean(ate1), np.mean(ate2)])    
    mse_ate = np.array([mse(ATE1, ate1), mse(ATE2, ate2)])
    bias_ate = np.array([np.sum(np.subtract(ATE1,ate1))/reps, np.sum(np.subtract(ATE2,ate2))/reps])
    var_ate = np.array([np.var(ate1), np.var(ate2)])

    #TOTAL MSE's
    ind_var = np.var(tauhat[:, 1:], axis = 1)
    ind_mean = np.mean(tauhat, axis = 1)

    total_var = np.sum(ind_var)/tsize
    total_bias = np.sum(np.square(np.subtract(ind_mean, tau)))/tsize
    total_mse = total_var + total_bias

    results = np.concatenate([np.array([nomin_test]), mean_ate, bias_ate, var_ate, mse_ate, np.array([total_bias]), np.array([total_var]), np.array([total_mse])])
    return results

In [ ]:
n = 500
tsize = 3000
reps = 500
test_sizes = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
res = []
var = [0.01, 1.0, 2.5]

#print("start time: ", str(datetime.datetime.now())[11:16])

for v in var:
    df = pd.DataFrame()
    for size in test_sizes:
        results = mc_ate2_test(n, tsize, v, reps, size)
        df = pd.concat([df, pd.DataFrame(results).transpose()])
    df.columns=['test_sizes', 'MSE_T_BIAS', 'MSE_T_VAR', 'MSE_TOTAL']
    res.append([v])
    res.append([df])
    
#print("end time: ", str(datetime.datetime.now())[11:16])

In [9]:
res

[[0.01],
 [   test_sizes  MSE_T_BIAS  MSE_T_VAR  MSE_TOTAL
  0         0.2    0.136339   0.506717   0.643056
  0         0.3    0.075862   0.422587   0.498449
  0         0.4    0.086525   0.247190   0.333715
  0         0.5    0.076038   0.248323   0.324360
  0         0.6    0.130646   0.337726   0.468372
  0         0.7    0.083239   0.237470   0.320709
  0         0.8    0.280659   0.452722   0.733381
  0         0.9    0.635079   0.674585   1.309664],
 [1.0],
 [   test_sizes  MSE_T_BIAS  MSE_T_VAR  MSE_TOTAL
  0         0.2    0.198980   0.876547   1.075527
  0         0.3    0.113098   0.316457   0.429556
  0         0.4    0.081825   0.276358   0.358182
  0         0.5    0.085708   0.288292   0.374000
  0         0.6    0.198010   0.207437   0.405447
  0         0.7    0.196531   0.373973   0.570504
  0         0.8    0.159501   0.317892   0.477394
  0         0.9    0.442942   0.612952   1.055894],
 [2.5],
 [   test_sizes  MSE_T_BIAS  MSE_T_VAR  MSE_TOTAL
  0         0.2    0.